In [1]:
import firebase_admin
from firebase_admin import credentials
from firebase_admin import db
from firebase_admin import storage
import hmm_predict

C:\Users\ACER\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [2]:
cred = credentials.Certificate("pbl5shsr-firebase-adminsdk-u4jih-5de6689dfa.json")

In [3]:
firebase_admin.initialize_app(cred, {
    'databaseURL': 'https://pbl5shsr-default-rtdb.asia-southeast1.firebasedatabase.app',
    'storageBucket': 'gs://pbl5shsr.appspot.com'
})


In [4]:
# Lấy reference đến node "devices"
devices_ref = db.reference('devices')

# Lấy dữ liệu từ node "devices"
devices_data = devices_ref.get()

# In ra dữ liệu
print(devices_data)

{'device1': {'name': 'Light', 'status': 0}, 'device2': {'name': 'Door', 'status': 1}, 'device3': {'name': 'Air conditioner', 'status': 1}, 'device4': {'name': 'Fan', 'speed': 1, 'status': 2}}


In [5]:
trangthai_ref = devices_ref.child('device1')

# Lấy giá trị của trường 'trangthai'
trangthai_val = trangthai_ref.get()

In [6]:
print(trangthai_val)

{'name': 'Light', 'status': 0}


In [7]:
status_ref = trangthai_ref.child('name')

# Lấy giá trị của trường 'trangthai'
status_val = status_ref.get()
print(status_val)

Light


In [11]:
# def on_data_change(event):
#     print('Data changed:', event.data)

# devices_ref.child('device1').child('status').listen(on_data_change)

# # Lặp vô hạn để tiếp tục lắng nghe sự thay đổi của Firebase Realtime Database
# while True:
#     pass

In [8]:
audios_ref = db.reference('audio')

In [9]:
bucket = storage.bucket()

In [10]:
file_ref = audios_ref.child('file_url')

In [11]:
file_urllink=file_ref.get()
print(file_urllink)

https://firebasestorage.googleapis.com/v0/b/pbl5shsr.appspot.com/o/audio%2FrecordingAudio.wav?alt=media&token=d878bf18-b7d0-42e8-bf44-487b3a0cdf46


In [12]:
import requests

In [13]:
response = requests.get(file_urllink)

with open("new_audio.wav", "wb") as f:
    f.write(response.content)

In [15]:
from pydub.playback import play
from pydub import AudioSegment
def on_data_change(event):
    print('Data changed:', event.data)
    file_urllink=file_ref.get()
    print(file_urllink)
    response = requests.get(file_urllink)

    with open("new_audio.wav", "wb") as f:
     f.write(response.content)
    
    from pydub.playback import play

    # Load file
    sound = AudioSegment.from_wav('new_audio.wav')

    # Play sound
    play(sound)
    hmm_predict.predict()
audios_ref.child('file_url').listen(on_data_change)

# Lặp vô hạn để tiếp tục lắng nghe sự thay đổi của Firebase Realtime Database
while True:
    pass

Data changed: https://firebasestorage.googleapis.com/v0/b/pbl5shsr.appspot.com/o/audio%2FrecordingAudio.wav?alt=media&token=fab72f39-1210-4da2-8bd4-3a8a9942b9c2
https://firebasestorage.googleapis.com/v0/b/pbl5shsr.appspot.com/o/audio%2FrecordingAudio.wav?alt=media&token=fab72f39-1210-4da2-8bd4-3a8a9942b9c2


KeyboardInterrupt: 

In [16]:
import modelcnn
modelcnn.predict()

reading audio file...
audio file read!
1/1 [==============================] - 0s 234ms/step
Predicted label: mocua


'mocua'

In [17]:
hmm_predict.predict()

scores [-10198.336700661028, -10401.008087886488, -9919.609321784483, -10263.372814470415, -9376.144020278041, -9869.566944943583, -10018.951949124608, -10529.432671429771, -10220.548113351097, -9951.666303327767]
mocua
scores [-6623.490692895, -6623.060732714532, -6310.5548945313585, -6491.131803713678, -5717.974781378913, -6147.5981906197, -6359.204217204515, -6521.123530709681, -6408.600232143468, -6284.972480689755]
mocua
scores [-6640.397418256823, -6633.927319799147, -6324.610935778282, -6505.6468725561845, -5736.032612587694, -6164.115957229736, -6372.807630873362, -6539.677878705331, -6419.424978247214, -6299.40270831156]
mocua


In [22]:

import pickle
import wave

import numpy as np
import pyaudio
from pydub import AudioSegment
import module.preprocessing as preprocessing






In [35]:
from pydub import AudioSegment





from pydub.playback import play

# Load file
sound = AudioSegment.from_wav('new_audio.wav')

# Play sound
play(sound)


In [28]:

class HMMRecognition:
    def __init__(self):
        self.model = {}

        self.class_names = ['batden', 'tatden', 'batquat','tatquat','mocua','dongcua']
        self.audio_format = 'wav'

        self.record_path = 'module/temp/record.wav'
        self.trimmed_path = 'module/temp/trimmed.wav'
        self.model_path = 'module/models_train'

        self.load_model()

    @staticmethod
    def detect_leading_silence(sound, silence_threshold=-42.0, chunk_size=10):
        trim_ms = 0  # ms

        assert chunk_size > 0  # to avoid infinite loop
        while sound[trim_ms:trim_ms + chunk_size].dBFS < silence_threshold and trim_ms < len(sound):
            trim_ms += chunk_size

        return trim_ms
    def load_model(self):
            for key in self.class_names:
                name = f"{self.model_path}/model_{key}.pkl"
                with open(name, 'rb') as file:
                    self.model[key] = pickle.load(file)
    def predict(self, file_name=None):
        if not file_name:
            file_name = self.record_path

        # Trim silence
        sound = AudioSegment.from_file(file_name, format=self.audio_format)

        start_trim = self.detect_leading_silence(sound)
        end_trim = self.detect_leading_silence(sound.reverse())

        duration = len(sound)

        trimmed_sound = sound[start_trim:duration - end_trim]
        trimmed_sound.export(self.trimmed_path, format=self.audio_format)

        # Predict
        record_mfcc = preprocessing.get_mfcc(self.trimmed_path)
        scores = [self.model[cname].score(record_mfcc) for cname in self.class_names]
        print('scores', scores)
        predict_word = np.argmax(scores)
        print(self.class_names[predict_word])
        string = self.class_names[predict_word]
        #update db
        if "quat" in string:
            refup = db.reference('devices/device4')
            if "tat" in string:
                refup.update({'status': '0'})
            elif "bat" in string:
                refup.update({'status': '1'})
            else: 
                refup.update({'status': '2'})
        elif "cua" in string:
            refup = db.reference('devices/device2')
            if "tat" in string:
                refup.update({'status': '0'})
            elif "bat" in string:
                refup.update({'status': '1'})
            else: 
                refup.update({'status': '1'})
        elif "den" in string:
            refup = db.reference('devices/device1')
            if "tat" in string:
                refup.update({'status': '0'})
            elif "bat" in string:
                refup.update({'status': '1'})
            else: 
                refup.update({'status': '1'})
        elif "dieuhoa" in string:
            refup = db.reference('devices/device3')
            if "tat" in string:
                refup.update({'status': '0'})
            elif "bat" in string:
                refup.update({'status': '1'})
            else: 
                refup.update({'status': '1'})    
        else:
            print('not found')
    def record(self):
        CHUNK = 1024
        FORMAT = pyaudio.paInt16
        CHANNELS = 1
        RATE = 16000
        RECORD_SECONDS = 6

        p = pyaudio.PyAudio()

        

        frames = []

        
        print('reading audio file...')
        with wave.open('audio.wav', 'rb') as wf:
            for i in range(0, int(RATE / CHUNK * RECORD_SECONDS)):
                data = wf.readframes(CHUNK)
                frames.append(data)

        

        print('audio file read!')
        
        p.terminate()

        wf = wave.open(self.record_path, 'wb')
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
        wf.close()

In [29]:
hmm_reg = HMMRecognition()
hmm_reg.record()
hmm_reg.predict()

reading audio file...
audio file read!
scores [-6946.472328083157, -7093.413202709599, -6807.787174007189, -6665.23313428039, -7198.763181441223, -7027.707489786817]
tatquat


Data changed: https://firebasestorage.googleapis.com/v0/b/pbl5shsr.appspot.com/o/audio%2FrecordingAudio.wav?alt=media&token=dbc5f40e-813f-4030-a1c0-0cf8576ab786
Data changed: https://firebasestorage.googleapis.com/v0/b/pbl5shsr.appspot.com/o/audio%2FrecordingAudio.wav?alt=media&token=dbc5f40e-813f-4030-a1c0-0cf8576ab786
Data changed: https://firebasestorage.googleapis.com/v0/b/pbl5shsr.appspot.com/o/audio%2FrecordingAudio.wav?alt=media&token=dbc5f40e-813f-4030-a1c0-0cf8576ab786
https://firebasestorage.googleapis.com/v0/b/pbl5shsr.appspot.com/o/audio%2FrecordingAudio.wav?alt=media&token=dbc5f40e-813f-4030-a1c0-0cf8576ab786
https://firebasestorage.googleapis.com/v0/b/pbl5shsr.appspot.com/o/audio%2FrecordingAudio.wav?alt=media&token=dbc5f40e-813f-4030-a1c0-0cf8576ab786
https://firebasestorage.googleapis.com/v0/b/pbl5shsr.appspot.com/o/audio%2FrecordingAudio.wav?alt=media&token=dbc5f40e-813f-4030-a1c0-0cf8576ab786


In [ ]:
string = "Helloworld zo"
if "Hello" in string:
    print("Found")
else:
    print("Not found")

Found
